# Capstone Project 1: Video Game Sales

## Data Cleaning and Wrangling:
The data for this capstone project came from a Kaggle competition: 
    https://www.kaggle.com/rush4ratio/video-game-sales-with-ratings
The data provided by this competition will be used to find insights into whether or not metacritic ratings have an affect on video game sales. 

The dataset below will be imported into a pandas DataFrame as well as converting all the 'tbd' values in NaNs.

In [1]:
#The data will be imported and inspected
#turning all the 'tbd' string into NaNs
import pandas as pd
import numpy as np
FILENAME = 'data\Video_Games_Sales_as_at_22_Dec_2016.csv'
df= pd.read_csv(FILENAME, na_values='tbd')
df.head(3)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E


The dataset will now be inspected to see what needs to be cleaned or dealt with.

In [2]:
#inspecting the columns
df.columns

Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating'],
      dtype='object')

Looking at the DataFrame info revealed that there are many null values in some of the columns.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
Name               16717 non-null object
Platform           16719 non-null object
Year_of_Release    16450 non-null float64
Genre              16717 non-null object
Publisher          16665 non-null object
NA_Sales           16719 non-null float64
EU_Sales           16719 non-null float64
JP_Sales           16719 non-null float64
Other_Sales        16719 non-null float64
Global_Sales       16719 non-null float64
Critic_Score       8137 non-null float64
Critic_Count       8137 non-null float64
User_Score         7590 non-null float64
User_Count         7590 non-null float64
Developer          10096 non-null object
Rating             9950 non-null object
dtypes: float64(10), object(6)
memory usage: 2.0+ MB


It looks like this dataset contains games that have yet to be released. This dataset was compiled in 2016. Any games past 2016 will be filtered out.

In [4]:
df.Year_of_Release.max()

2020.0

In [5]:
#Filtering for games only up to 2016
df_years = df[df['Year_of_Release'] < 2017]
df_years['Year_of_Release'].max()

2016.0

Given the main question of this capstone project, only records that have both critic and user scores and have global sales larger than 0 will be observed. All other video games will not be included.

In [19]:
#will only want to work with data that has critic/user scores and Global sales > 0
dfScores = df_years[(df_years['User_Score'].notnull() &
                     df_years['Critic_Score'].notnull()) &
                    df_years['Global_Sales'] > 0]
dfScores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6894 entries, 0 to 16709
Data columns (total 16 columns):
Name               6894 non-null object
Platform           6894 non-null object
Year_of_Release    6894 non-null float64
Genre              6894 non-null object
Publisher          6893 non-null object
NA_Sales           6894 non-null float64
EU_Sales           6894 non-null float64
JP_Sales           6894 non-null float64
Other_Sales        6894 non-null float64
Global_Sales       6894 non-null float64
Critic_Score       6894 non-null float64
Critic_Count       6894 non-null float64
User_Score         6894 non-null float64
User_Count         6894 non-null float64
Developer          6890 non-null object
Rating             6826 non-null object
dtypes: float64(10), object(6)
memory usage: 915.6+ KB


The Year_of_Release column currently contains values that have decimal places. Years are not expressed as floats. This column will be converted into a integer data type.

In [20]:
#converting the year from float64 to int64
dfScores.loc[:,'Year_of_Release'] = dfScores.loc[:,'Year_of_Release'].astype('int64')
dfScores.Year_of_Release.head()

0    2006
2    2008
3    2009
6    2006
7    2006
Name: Year_of_Release, dtype: int64

Using the pandas DataFrame .describe method, the summary statistics of all the numerical columns can be observed.

In [21]:
#Observing summary statistics
dfScores.describe()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count
count,6894.000000,6894.000000,6894.000000,6894.000000,6894.00000,6894.000000,6894.000000,6894.000000,6894.000000,6894.000000
mean,2007.482303,0.390920,0.234517,0.063867,0.08200,0.771487,70.258486,28.842472,7.184378,174.392370
std,4.236401,0.963231,0.684214,0.286461,0.26862,1.954780,13.861082,19.194572,1.439806,584.872155
min,1985.000000,0.000000,0.000000,0.000000,0.00000,0.010000,13.000000,3.000000,0.500000,4.000000
25%,2004.000000,0.060000,0.020000,0.000000,0.01000,0.110000,62.000000,14.000000,6.500000,11.000000
50%,2007.000000,0.150000,0.060000,0.000000,0.02000,0.290000,72.000000,24.000000,7.500000,27.000000
75%,2011.000000,0.390000,0.210000,0.010000,0.07000,0.750000,80.000000,39.000000,8.200000,89.000000
max,2016.000000,41.360000,28.960000,6.500000,10.57000,82.530000,98.000000,113.000000,9.600000,10665.000000


There is a very large outlier in global sales. Sorting the global sales columns reveals that this outlier comes from "Wii Sports". This outlier will be kept in the dataset for now.

In [22]:
dfScores[['Name', 'Global_Sales']].sort_values('Global_Sales', ascending=False).head()

,Name,Global_Sales
0,Wii Sports,82.53
2,Mario Kart Wii,35.52
3,Wii Sports Resort,32.77
6,New Super Mario Bros.,29.80
7,Wii Play,28.92


Using the value_counts method, it is seen that there are many repeats for some video game titles.

In [23]:
#check for duplicates
dfScores.Name.value_counts().head()

Need for Speed: Most Wanted                8
Madden NFL 07                              8
LEGO Star Wars II: The Original Trilogy    8
Need for Speed Carbon                      7
Madden NFL 08                              7
Name: Name, dtype: int64

Looking deeper as to why certain games appear multiple times. A high value counted video game, Need for Speed, was looked at. It is seen that the reason for the multiple occurrences was due to the video game being released on different platforms and for different years. It is known that some video games are re-released at a later date.

In [45]:
#inspecting as to why there are duplicate names
df_NfS = dfScores[dfScores.Name == 'Need for Speed: Most Wanted'].sort_values('Global_Sales', ascending=False)
df_NfS.head(3)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
253,Need for Speed: Most Wanted,PS2,2005,Racing,Electronic Arts,2.03,1.79,0.08,0.47,4.37,82.0,36.0,9.1,137.0,EA Canada,T
1190,Need for Speed: Most Wanted,X360,2012,Racing,Electronic Arts,0.62,0.78,0.01,0.15,1.56,83.0,54.0,8.5,134.0,EA Canada,T
1591,Need for Speed: Most Wanted,X360,2005,Racing,Electronic Arts,1.00,0.13,0.02,0.10,1.25,83.0,54.0,8.5,134.0,EA Canada,T


To deal with the duplicates the dataset will need to be grouped by the video game name. The rest of the columns would need to be aggregated. The year with the largest sales, the publisher with the most occurrences, the sum of the global sales, max of the critic and user scores, and the mode of the genre will be computed.

In [62]:
#will now apply the aggregation to the dataset with the relevant columns
dfScores.sort_values('Global_Sales', ascending=False, inplace=True)
columns = ['Name', 'Year_of_Release','Publisher', 'Genre', 'Global_Sales',
           'Critic_Score', 'User_Score']
dfClean = dfScores.loc[:,columns].groupby('Name').agg({'Genre':pd.Series.mode,
                                                   'Year_of_Release':'first',
                                                   'Publisher': pd.Series.mode,
                                                   'Global_Sales': 'sum',
                                                   'Critic_Score': 'max',
                                                   'User_Score': 'max'              
                                                 })
dfClean.head(3)

C:\Users\Thompson\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Genre,Year_of_Release,Publisher,Global_Sales,Critic_Score,User_Score
Name,,,,,,
Tales of Xillia 2,Role-Playing,2012,Namco Bandai Games,0.84,71.0,7.9
.hack//Infection Part 1,Role-Playing,2002,Atari,1.27,75.0,8.5
.hack//Mutation Part 2,Role-Playing,2002,Atari,0.68,76.0,8.9


The final data cleaning steps will be to scale up the user scores to match the critic scores and to sort the dataset by global sales. Scaling up the user scores will allow comparisons to critic scores with the other variables.

In [63]:
#Scaling up user scores and sorting the dataset by global sales
dfClean['User_Score'] = dfClean['User_Score'] * 10
dfClean = dfClean.sort_values('Global_Sales', ascending=False).reset_index()
dfClean.head()

,Name,Genre,Year_of_Release,Publisher,Global_Sales,Critic_Score,User_Score
0,Wii Sports,Sports,2006,Nintendo,82.53,76.0,80.0
1,Grand Theft Auto V,Action,2013,Take-Two Interactive,56.57,97.0,83.0
2,Mario Kart Wii,Racing,2008,Nintendo,35.52,82.0,83.0
3,Wii Sports Resort,Sports,2009,Nintendo,32.77,80.0,80.0
4,Call of Duty: Modern Warfare 3,Shooter,2011,Activision,30.59,88.0,34.0


After the cleaning and data wrangling of the video games dataset, the total amount of video games that will be looked at went from 16.7 thousand to about 4.4 thousand. This cleaned dataset will be exported to a csv in order to perform the next steps of this capstone project.

In [64]:
#final checks of the dataset
dfClean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4428 entries, 0 to 4427
Data columns (total 7 columns):
Name               4428 non-null object
Genre              4428 non-null object
Year_of_Release    4428 non-null int64
Publisher          4428 non-null object
Global_Sales       4428 non-null float64
Critic_Score       4428 non-null float64
User_Score         4428 non-null float64
dtypes: float64(3), int64(1), object(3)
memory usage: 242.2+ KB


In [65]:
dfClean.describe()

,Year_of_Release,Global_Sales,Critic_Score,User_Score
count,4428.000000,4428.000000,4428.000000,4428.000000
mean,2007.011292,1.201136,70.453704,74.336721
std,4.294223,2.877575,14.049812,13.668565
min,1985.000000,0.010000,19.000000,7.000000
25%,2004.000000,0.130000,62.000000,68.000000
50%,2007.000000,0.380000,73.000000,78.000000
75%,2010.000000,1.170000,81.000000,84.000000
max,2016.000000,82.530000,98.000000,96.000000


In [66]:
#Exporting Data to csv
dfClean.to_csv("Clean_Video_Game_Sales.csv")